In [ ]:
import hashlib
import binascii

# Logical functions used by SHA-256
def rotr(n, x):
    return (x >> n) | (x << (32 - n)) & 0xFFFFFFFF

def shr(n, x):
    return x >> n

def ch(x, y, z):
    return (x & y) ^ (~x & z)

def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

def sigma0(x):
    return rotr(2, x) ^ rotr(13, x) ^ rotr(22, x)

def sigma1(x):
    return rotr(6, x) ^ rotr(11, x) ^ rotr(25, x)

def gamma0(x):
    return rotr(7, x) ^ rotr(18, x) ^ shr(3, x)

def gamma1(x):
    return rotr(17, x) ^ rotr(19, x) ^ shr(10, x)

# SHA-256 constants
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# SHA-256 main algorithm
def sha256(message):
    # Preprocessing
    message = bytearray(message)  # Convert to byte array
    orig_len_in_bits = (8 * len(message)) & 0xFFFFFFFFFFFFFFFF
    message.append(0x80)
    
    while len(message) % 64 != 56:
        message.append(0)
    
    message += orig_len_in_bits.to_bytes(8, byteorder='big')
    
    hash_pieces = [
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    ]
    
    # Process the message in successive 512-bit chunks
    for i in range(0, len(message), 64):
        w = [0] * 64
        w[0:16] = [int.from_bytes(message[i:i+4], 'big') for i in range(0, 64, 4)]
        
        for j in range(16, 64):
            w[j] = (gamma1(w[j-2]) + w[j-7] + gamma0(w[j-15]) + w[j-16]) & 0xFFFFFFFF
        
        a, b, c, d, e, f, g, h = hash_pieces
        
        for j in range(64):
            t1 = (h + sigma1(e) + ch(e, f, g) + K[j] + w[j]) & 0xFFFFFFFF
            t2 = (sigma0(a) + maj(a, b, c)) & 0xFFFFFFFF
            h = g
            g = f
            f = e
            e = (d + t1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (t1 + t2) & 0xFFFFFFFF
        
        # Add the compressed chunk to the current hash value
        hash_pieces = [(x + y) & 0xFFFFFFFF for x, y in zip(hash_pieces, [a, b, c, d, e, f, g, h])]
    
    # Produce the final hash value (big-endian)
    return sum(x << (32 * i) for i, x in enumerate(reversed(hash_pieces)))

# Example usage
message = b"Hello, world!"
print(binascii.hexlify(sha256(message).to_bytes(32, byteorder='big')).decode())